In [6]:
## Suppose we have an unsorted list:
my_list = [3, 5, 2, 4, 9, 5, 7, 0, 10]

""" Objective: Find #7
Modify the classical search approach (O(N) linear traversal) by defining an oracle (black box)
"""
## (Black box to produce a solution for any instance of a given problem at our disposal)
def the_oracle(my_input):
    winner=7
    if(my_input is winner):
        return True
    return False

In [12]:
# How many times will we need to call the oracle to get an actual winner?

for index, trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print('Winner found at index %i' %index)
        print('%i calls to the Oracle used' %(index + 1))
        break

Winner found at index 6
7 calls to the Oracle used


## Quantum Oracle: Flips the sign if the input state matches the "winner" (11) state

Grover's algorithm turns the classical algorithm into sqrt(N)

In [10]:
## Controlled Z-gate flips the state of the |11> state
## Also need amplitude amplficiation (reflection operator)
## Grover's operator = amplification + oracle

In [9]:
from qiskit import *
import matplotlib.pyplot as plot
import numpy as np

In [11]:
#define a circuit to act as an oracle
oracle = QuantumCircuit(2, name='oracle')
## Controlled not gate
oracle.cz(0, 1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [12]:

## Check Oracle functionality by preparing superposition state of all qubits by applying hadamard gate
from qiskit_ibm_runtime import QiskitRuntimeService
backend_name = 'ibm_brisbane'
service = QiskitRuntimeService()
backend = service.get_backend(backend_name)
## Two qubits and two classical registers
grover_circ = QuantumCircuit(2, 2)
## Add hadamard gate on both qubits to prepare all 4 superposition states
grover_circ.h([0, 1])
## Add oracle to query both superpositioned states at the same time
grover_circ.append(oracle, [0, 1])
grover_circ.measure([0, 1], [0, 1])
grover_circ.draw()

┌───┐┌─────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤M├───
     ├───┤│  oracle │└╥┘┌─┐
q_1: ┤ H ├┤1        ├─╫─┤M├
     └───┘└─────────┘ ║ └╥┘
c: 2/═════════════════╩══╩═
                      0  1

In [14]:
from qiskit import transpile
 
new_circuit = transpile(grover_circ, backend)
job = backend.run(new_circuit)
result = job.result()

C:\Users\ejfei\AppData\Local\Temp\ipykernel_1736\1647404501.py:4: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(new_circuit)


IBMBackendApiError: 'Error submitting job: \'409 Client Error: Conflict for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"You have reached the limit of 3 pending  jobs. Please wait for a job to complete or cancel one before submitting anything new.","code":3458,"solution":"Wait until some previous jobs were finished. You can cancel pending jobs to run new jobs.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [ ]:
sv = result.get_statevector()
## Prepared superposition state and got same states except with the (1,1) state flipped
np.around(sv, 2)

In [2]:
""" Must perform amplitude amplification to amplify the probabilities of the winning state and reduce probability of non-winning states
Create new state in the span of the winning state and the superposition state called s' which is orthogonal to the winning state

s' is the superposition state without the winning components

Applying the oracle operator to the superpositioned state will flip the sign of the winning state
    (Geometrically, we're reflecting s around s'): |s><s| - I
"""
from qiskit import QuantumCircuit

# Create the reflection circuit
reflection = QuantumCircuit(2, name='reflection')

# Add Hadamard gates to all qubits to create superposition
reflection.h([0, 1])

# Apply Z gate on both qubits
reflection.z([0, 1])

# Apply controlled-Z gate
reflection.cz(0, 1)

# Apply Hadamard gates to all qubits to bring back to original state
reflection.h([0, 1])

# Convert to gate
reflection_gate = reflection.to_gate()

# Optionally, print the circuit for visualization
print(reflection)

     ┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘


In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
backend_name = 'ibm_brisbane'
service = QiskitRuntimeService()
backend = service.get_backend(backend_name)
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.append(reflection, [0, 1])
grover_circ.measure([0, 1], [0, 1])

In [7]:
# Prepared superposition state
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [17]:
from qiskit import transpile
## After having the oracle and reflection operators, we want the state |11>
new_circuit = transpile(grover_circ, backend)
job = backend.run(new_circuit)
result = job.result()
## Should give back {11 state}
result.get_counts()

C:\Users\ejfei\AppData\Local\Temp\ipykernel_1736\1294120368.py:4: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(new_circuit)
